In [1]:
from PIL import Image
import re
import csv
import pandas as pd
import numpy as np
import tabula
import glob

import cv2
import numpy as np


import aircv as ac
import pyautogui
import matplotlib.pyplot as plt


import sys
import fitz

import shutil
import os 
import math

from functools import cmp_to_key

In [2]:
def remove_red(img_cv):
    # Convert BGR to RGB
    img_rgb = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)

    # Convert the image to HSV
    img_hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)

    # Define a wide range around the red color in HSV space, split into two parts
    lower_red1 = np.array([0, 30, 100])
    upper_red1 = np.array([30, 255, 255])

    lower_red2 = np.array([150, 30, 100])
    upper_red2 = np.array([180, 255, 255])

    # Threshold the HSV image to get only red colors
    mask1 = cv2.inRange(img_hsv, lower_red1, upper_red1)
    mask2 = cv2.inRange(img_hsv, lower_red2, upper_red2)

    # Combine the masks
    mask = mask1 | mask2

    # Replace all red (now white in the mask) pixels with white in the original image
    img_rgb[mask > 0] = [255, 255, 255]

    img_rgb = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)

    return img_rgb

def find_square_corners(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 進行二值化處理（根據圖像特性選擇適合的閾值方法）
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    """
    plt.figure(figsize=(10,10))
    plt.imshow(cv2.cvtColor(binary, cv2.COLOR_BGR2RGB))
    plt.show()
    """

    # 使用形態學操作來擴展網格線（根據圖像特性選擇適合的核大小）
    kernel = np.ones((7, 7), np.uint8)
    dilated = cv2.dilate(binary, kernel, iterations=1)

    """
    plt.figure(figsize=(10,10))
    plt.imshow(cv2.cvtColor(dilated, cv2.COLOR_BGR2RGB))
    plt.show()
    """

    # Find contours in the edge map
    cnts, _ = cv2.findContours(dilated.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Initialize the corners
    squares = []

    # 遍歷輪廓，繪製矩形框
    min_area_threshold = 50000  # 設定最小面積閾值
    max_area_threshold = 1000000  # 設定最大面積閾值

    # Loop over the contours
    for c in cnts:
        # Approximate the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.045 * peri, True)

        # If our approximated contour has four points, we can assume that we have found a grid
        if len(approx) == 4:
            # Check if all angles are approximately 90 degrees
            x, y, w, h = cv2.boundingRect(c)
            area = w * h  # 計算矩形框的面積
            if area > min_area_threshold and area < max_area_threshold: #超過一定的面積
                cos = []
                for i in range(2, 5):
                    cos.append(angle(approx[i%4], approx[i-2], approx[i-1]))
                    cos.sort()
                    ratio = w / float(h)  # assuming h != 0
                    if 0.8 <= ratio <= 1.2:  # checks if w and h are within 90% of each other
                        squares.append(approx)
                    
    return squares

# Function to return needed angle
def angle(pt1, pt2, pt0):
    dx1 = pt1[0][0] - pt0[0][0]
    dy1 = pt1[0][1] - pt0[0][1]
    dx2 = pt2[0][0] - pt0[0][0]
    dy2 = pt2[0][1] - pt0[0][1]
    return (dx1*dx2 + dy1*dy2)/np.sqrt((dx1*dx1 + dy1*dy1)*(dx2*dx2 + dy2*dy2) + 1e-10)
# 去除重疊的函式
def remove_overlapping_squares(squares, overlap_threshold=0.8):
    # Calculate the bounding rectangles of the squares
    rects = [cv2.boundingRect(square) for square in squares]

    # Initialize a list to hold the non-overlapping squares
    non_overlapping_squares = []

    # Loop over the squares
    for i in range(len(squares)):
        # Assume that the square does not overlap with any other square
        overlaps = False

        # Calculate the area of the square
        area1 = rects[i][2] * rects[i][3]

        # Loop over the other squares
        for j in range(i + 1, len(squares)):
            # Calculate the intersection of the two bounding rectangles
            x = max(rects[i][0], rects[j][0])
            y = max(rects[i][1], rects[j][1])
            w = min(rects[i][0] + rects[i][2], rects[j][0] + rects[j][2]) - x
            h = min(rects[i][1] + rects[i][3], rects[j][1] + rects[j][3]) - y

            # If there is an intersection, calculate the area of the intersection
            if w > 0 and h > 0:
                intersection_area = w * h

                # Calculate the area of the other square
                area2 = rects[j][2] * rects[j][3]

                # If the area of the intersection is greater than the threshold for either square, mark the square as overlapping
                if intersection_area > overlap_threshold * area1 or intersection_area > overlap_threshold * area2:
                    overlaps = True
                    break

        # If the square does not overlap with any other square, add it to the list of non-overlapping squares
        if not overlaps:
            non_overlapping_squares.append(squares[i])

    return non_overlapping_squares

def mask(image):
    # Create a copy of the original image
    output = image.copy()

    # Define the rectangle
    x1, y1, x2, y2 = 316, 838, 880, 1405

    # Create a black mask with the same dimensions as the image
    mask = np.zeros_like(image)

    # Set the region inside the rectangle to white
    mask[y1:y2, x1:x2] = [255, 255, 255]

    # Split the mask and the image into their respective channels
    mask_b, mask_g, mask_r = cv2.split(mask)
    output_b, output_g, output_r = cv2.split(output)

    # Use the mask to change all black (now white in the mask) pixels to white in the original image
    output_b[mask_b == 0] = 255
    output_g[mask_g == 0] = 255
    output_r[mask_r == 0] = 255

    # Merge the channels back into a single image
    output = cv2.merge([output_b, output_g, output_r])

    return output

# Function to draw squares on the image
def draw_squares(image, squares):
    for square in squares:
        cv2.drawContours(image, [square], -1, (0, 255, 0), 3)

def order_points(pts):
    # 重新塑形點，以便我們可以使用 numpy 的函數
    pts = pts.reshape((4,2))

    # 分別計算點的 x 和 y 坐標的總和和差
    s = pts.sum(axis=1)
    d = np.subtract(pts[:, 0], pts[:, 1])

    # 左上角的點具有最小的 x + y 值
    tl = pts[np.argmin(s)]
    # 右上角的點具有最大的 x - y 值
    tr = pts[np.argmax(d)]
    # 右下角的點具有最大的 x + y 值
    br = pts[np.argmax(s)]
    # 左下角的點具有最小的 x - y 值
    bl = pts[np.argmin(d)]

    return np.array([tl, tr, br, bl], dtype="float32")


# 定義比較函數
def compare_y(square1, square2):
    center1 = square1.mean(axis=0)
    center2 = square2.mean(axis=0)
    return -1 if center1[1] > center2[1] else 1

def compare_x(square1, square2):
    center1 = square1.mean(axis=0)
    center2 = square2.mean(axis=0)
    return -1 if center1[0] > center2[0] else 1  



In [3]:
# Load the Excel file
df = pd.read_excel('../智力測驗總表V2.xlsx')
df_t = df.transpose()
score_list = []

pdf_to_jpg_allfile = glob.glob('../classification_doc_v3/clear_far43/*.jpg')

print(pdf_to_jpg_allfile)
print(len(pdf_to_jpg_allfile))

imges43list = [] #準備要存43個(25字)

for i in range(len(pdf_to_jpg_allfile)):
    #load 智力
    number = int(pdf_to_jpg_allfile[i][-13:-7])
    scores = df_t.loc[number]
    score_list.append(np.array(scores))

    # Load the image
    image = cv2.imread(pdf_to_jpg_allfile[i])
    image = remove_red(image)
    squares = find_square_corners(image)
    squares = remove_overlapping_squares(squares)
    """
    draw_squares(image, squares)
    plt.figure(figsize=(5,5))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.show()
    """
    #先把squares裡面的四個點依序左上、右上、右下、左下排序
    squares = [order_points(square) for square in squares]
    # 對y座標排序，再對x座標排序 讓位置是右上開始往下數，一直數到左下。
    squares = sorted(squares, key=cmp_to_key(compare_y))
    squares = sorted(squares, key=cmp_to_key(compare_x))

    src_pts = np.array([squares[0][0], squares[0][1], squares[0][2],squares[0][3]], dtype="float32")
    dst_pts = np.array([[0, 0], [500, 0], [500, 500], [0, 500]], dtype="float32") # we choose 300x400 as the size of the output image
    # Compute the perspective transformation matrix and then apply it
    matrix = cv2.getPerspectiveTransform(src_pts, dst_pts)
    warped = cv2.warpPerspective(image, matrix, (500, 500))

    img25list = [] #準備要存(25字)
    count = 0
    for j in range(5):
        for k in range(5):
            #cv2.imwrite(f"./2_6_2far_crop/{pdf_to_jpg_allfile[i][-13:-7]}({str(count).zfill(2)}).jpg",warped[100*j:100*(j+1),100*k:100*(k+1)])
            img25list.append(warped[100*j:100*(j+1),100*k:100*(k+1)])
            count +=1

    images25 = np.stack(img25list, axis=0)
    imges43list.append(images25)


images43 = np.stack(imges43list, axis=0)  
print(images43.shape)
scores43 = np.stack(score_list, axis=0)  
print(scores43.shape)


['../classification_doc_v3/clear_far43\\177090(1).jpg', '../classification_doc_v3/clear_far43\\210335(0).jpg', '../classification_doc_v3/clear_far43\\210621(1).jpg', '../classification_doc_v3/clear_far43\\225694(1).jpg', '../classification_doc_v3/clear_far43\\232047(4).jpg', '../classification_doc_v3/clear_far43\\259583(1).jpg', '../classification_doc_v3/clear_far43\\268926(1).jpg', '../classification_doc_v3/clear_far43\\268927(1).jpg', '../classification_doc_v3/clear_far43\\309535(4).jpg', '../classification_doc_v3/clear_far43\\312555(1).jpg', '../classification_doc_v3/clear_far43\\337556(1).jpg', '../classification_doc_v3/clear_far43\\342668(0).jpg', '../classification_doc_v3/clear_far43\\345573(1).jpg', '../classification_doc_v3/clear_far43\\346595(1).jpg', '../classification_doc_v3/clear_far43\\347685(1).jpg', '../classification_doc_v3/clear_far43\\348758(1).jpg', '../classification_doc_v3/clear_far43\\349736(1).jpg', '../classification_doc_v3/clear_far43\\349764(1).jpg', '../class

C:\Users\evan6\AppData\Local\Temp\ipykernel_29748\1067981338.py:90: RuntimeWarning: overflow encountered in scalar multiply
  return (dx1*dx2 + dy1*dy2)/np.sqrt((dx1*dx1 + dy1*dy1)*(dx2*dx2 + dy2*dy2) + 1e-10)
C:\Users\evan6\AppData\Local\Temp\ipykernel_29748\1067981338.py:90: RuntimeWarning: invalid value encountered in sqrt
  return (dx1*dx2 + dy1*dy2)/np.sqrt((dx1*dx1 + dy1*dy1)*(dx2*dx2 + dy2*dy2) + 1e-10)


(43, 25, 100, 100, 3)
(43, 5)


In [4]:
#要存再開
#np.save('4_1_x.npy', images43)
#np.save('4_1_y.npy', scores43)